<a href="https://colab.research.google.com/github/patty-13/DEEP_LEARNING_NJIT/blob/main/HW4_DEEPLEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW3 Image Classification


# Check GPU Type

In [ ]:
!nvidia-smi

Wed Sep 27 01:30:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Download Data


In [ ]:
!pip install gdown --upgrade
#!gdown --id '1DguOlR9s351nb6Z9p2V9PeoLg9EZuk_-' --output data.zip  # my link
!gdown --id '19ZlT0qm-3rdMRe60ya25xiN-ELOohr8M' --output data.zip
#https://drive.google.com/file/d/1DguOlR9s351nb6Z9p2V9PeoLg9EZuk_-/view?usp=sharing
#https://drive.google.com/file/d/1DguOlR9s351nb6Z9p2V9PeoLg9EZuk_-/view?usp=sharing
#'19ZlT0qm-3rdMRe60ya25xiN-ELOohr8M'

  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=19ZlT0qm-3rdMRe60ya25xiN-ELOohr8M
From (redirected): https://drive.google.com/uc?id=19ZlT0qm-3rdMRe60ya25xiN-ELOohr8M&confirm=t&uuid=bc39a496-0856-4a00-91c2-97d1de30afa1
To: /content/data.zip
100% 1.03G/1.03G [00:08<00:00, 123MB/s]


In [ ]:
! unzip data.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/train/5_1294.jpg   
  inflating: data/train/2_237.jpg    
  inflating: data/train/5_908.jpg    
  inflating: data/train/2_223.jpg    
  inflating: data/train/7_5.jpg      
  inflating: data/train/9_286.jpg    
  inflating: data/train/5_1280.jpg   
  inflating: data/train/10_97.jpg    
  inflating: data/train/6_161.jpg    
  inflating: data/train/3_756.jpg    
  inflating: data/train/2_545.jpg    
  inflating: data/train/0_695.jpg    
  inflating: data/train/3_811.jpg    
  inflating: data/train/2_1150.jpg   
  inflating: data/train/9_319.jpg    
  inflating: data/train/5_883.jpg    
  inflating: data/train/3_37.jpg     
  inflating: data/train/4_448.jpg    
  inflating: data/train/5_897.jpg    
  inflating: data/train/3_23.jpg     
  inflating: data/train/2_1144.jpg   
  inflating: data/train/8_678.jpg    
  inflating: data/train/3_805.jpg    
  inflating: data/train/5_129.jpg    
  inflating: data/train/9_443.jpg    

# Import Packages

In [ ]:
_exp_name = "sample"

In [ ]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm.auto import tqdm
import random
from sklearn.model_selection import KFold
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed = 1213
same_seeds(seed)

# Transforms
Torchvision provides lots of useful utilities for image preprocessing, data *wrapping* as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [ ]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
  #  Randomly apply horizontal flip with a 50% chance
   # transforms.RandomHorizontalFlip(),
    # # Randomly rotate the image by up to 10 degrees
   # transforms.RandomRotation(45),
    # # You may add some transforms here.
   # transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),  # Color jitter
    # random crop
   # transforms.RandomCrop((100, 100)),
    #transforms.GaussianBlur(23,1),
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])


# Datasets
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [ ]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files

        self.transform = tfm

    def __len__(self):
        return len(self.files)

    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)

        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label

        return im,label

# Model

In [ ]:
# class Classifier(nn.Module):
#     def __init__(self):
#         super(Classifier, self).__init__()
#         # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
#         # torch.nn.MaxPool2d(kernel_size, stride, padding)
#         # input dimension [3, 128, 128]
#         self.cnn = nn.Sequential(
#             nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

#             nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
#             nn.BatchNorm2d(128),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

#             nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
#             nn.BatchNorm2d(256),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

#             nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
#             nn.BatchNorm2d(512),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]

#             nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
#             nn.BatchNorm2d(512),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
#         )
#         self.fc = nn.Sequential(
#             nn.Linear(512*4*4, 1024),
#             nn.ReLU(),
#             nn.Dropout(p=0.5),
#             nn.Linear(1024, 512),
#             nn.ReLU(),
#             nn.Dropout(p=0.5),  # Added dropout here
#             nn.Linear(512, 11)  # Adjusted output size to match your task
#         )



#     def forward(self, x):
#         out = self.cnn(x)
#         out = out.view(out.size()[0], -1)
#         return self.fc(out)

import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self, num_classes=11):
        super(Classifier, self).__init__()
        # Define the convolutional layers
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),  # Batch normalization
            nn.ReLU(),
            # nn.Conv2d(64, 64, kernel_size=3, padding=1),
            # nn.BatchNorm2d(64),  # Batch normalization
            # nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),  # Batch normalization
            nn.ReLU(),
            # nn.Conv2d(128, 128, kernel_size=3, padding=1),
            # nn.BatchNorm2d(128),  # Batch normalization
            # nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),  # Batch normalization
            nn.ReLU(),
            # nn.Conv2d(256, 256, kernel_size=3, padding=1),
            # nn.BatchNorm2d(256),  # Batch normalization
            # nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),  # Batch normalization
            nn.ReLU(),
            # nn.Conv2d(512, 512, kernel_size=3, padding=1),
            # nn.BatchNorm2d(512),  # Batch normalization
            # nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),  # Batch normalization
            nn.ReLU(),
            # nn.Conv2d(512, 512, kernel_size=3, padding=1),
            # nn.BatchNorm2d(512),  # Batch normalization
            # nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Define the fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 4, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),  # Added dropout here
            nn.Linear(512, 11)  # Adjusted output size to match your task
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


# import torch.nn as nn

# class Classifier(nn.Module):
#     def __init__(self):
#         super(Classifier, self).__init__()
#         # VGG16-like architecture
#         self.cnn = nn.Sequential(
#             nn.Conv2d(3, 64, 3, padding=1),  # Conv1_1
#             nn.ReLU(inplace=True),
#             nn.Conv2d(64, 64, 3, padding=1),  # Conv1_2
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(2, stride=2),  # MaxPool1

#             nn.Conv2d(64, 128, 3, padding=1),  # Conv2_1
#             nn.ReLU(inplace=True),
#             nn.Conv2d(128, 128, 3, padding=1),  # Conv2_2
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(2, stride=2),  # MaxPool2

#             nn.Conv2d(128, 256, 3, padding=1),  # Conv3_1
#             nn.ReLU(inplace=True),
#             nn.Conv2d(256, 256, 3, padding=1),  # Conv3_2
#             nn.ReLU(inplace=True),
#             nn.Conv2d(256, 256, 3, padding=1),  # Conv3_3
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(2, stride=2),  # MaxPool3

#             nn.Conv2d(256, 512, 3, padding=1),  # Conv4_1
#             nn.ReLU(inplace=True),
#             nn.Conv2d(512, 512, 3, padding=1),  # Conv4_2
#             nn.ReLU(inplace=True),
#             nn.Conv2d(512, 512, 3, padding=1),  # Conv4_3
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(2, stride=2),  # MaxPool4

#             nn.Conv2d(512, 512, 3, padding=1),  # Conv5_1
#             nn.ReLU(inplace=True),
#             nn.Conv2d(512, 512, 3, padding=1),  # Conv5_2
#             nn.ReLU(inplace=True),
#             nn.Conv2d(512, 512, 3, padding=1),  # Conv5_3
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(2, stride=2)  # MaxPool5
#         )
#         self.fc = nn.Sequential(
#             nn.Linear(512 * 4 * 4, 1024),  # Adjust the input size based on your input image size
#             nn.ReLU(inplace=True),
#             nn.Dropout(p=0.5),
#             nn.Linear(1024, 1024),
#             nn.ReLU(inplace=True),
#             nn.Dropout(p=0.5),
#             nn.Linear(1024, 11)  # Output classes (adjust as needed)
#         )

#     def forward(self, x):
#         out = self.cnn(x)
#         out = out.view(out.size(0), -1)
#         out = self.fc(out)
#         return out







# Configurations

In [ ]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
#model = Classifier().to(device)

# The number of batch size.
batch_size = 64

# The number of training epochs.
n_epochs = 30

# If no improvement in 'patience' epochs, early stop.
patience = 4

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
#optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-6)

# Dataloader

In [ ]:
# Construct train and valid datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = FoodDataset("./data/train", tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = FoodDataset("./data/valid", tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

# Start Training

In [ ]:
# # Learning rate scheduler
# scheduler = ReduceLROnPlateau(optimizer, mode='max', patience=3, factor=0.5, verbose=True)

# # Define the number of folds for k-fold cross-validation
# stale = 0
# best_acc = 0


# from sklearn.model_selection import KFold

# # Define the number of folds for k-fold cross-validation
# n_splits = 3  # You can adjust this value as needed

# # Initialize k-fold cross-validation
# kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# # Define your model, optimizer, scheduler, and other parameters as you did before
# best_overall_acc = 0

# best_model_state_dict = None
# # Loop over each fold
# for fold, (train_index, valid_index) in enumerate(kf.split(train_set)):

#   print(f"Fold {fold + 1}/{n_splits}")

#   # Split the data into training and validation sets for this fold
#   train_subset = torch.utils.data.Subset(train_set, train_index)
#   valid_subset = torch.utils.data.Subset(train_set, valid_index)

#   train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
#   valid_loader = DataLoader(valid_subset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

  # Reset the model and other parameters for each fold
  model = Classifier().to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-2)
  #optimizer = torch.optim.SGD(model.parameters(), lr=0.0005, momentum=0.9, weight_decay=1e-4)  # Adjust lr, momentum, and weight_decay as needed
  scheduler = ReduceLROnPlateau(optimizer, mode='max', patience=3, factor=0.5, verbose=True)
  best_acc = 0
  stale = 0

  for epoch in range(n_epochs):

      # ---------- Training ----------
      # Make sure the model is in train mode before training.
      model.train()

      # These are used to record information in training.
      train_loss = []
      train_accs = []

      for batch in tqdm(train_loader):

          # A batch consists of image data and corresponding labels.
          imgs, labels = batch
          #imgs = imgs.half()
          #print(imgs.shape,labels.shape)

          # Forward the data. (Make sure data and model are on the same device.)
          logits = model(imgs.to(device))

          # Calculate the cross-entropy loss.
          # We don't need to apply softmax before computing cross-entropy as it is done automatically.
          loss = criterion(logits, labels.to(device))

          # Gradients stored in the parameters in the previous step should be cleared out first.
          optimizer.zero_grad()

          # Compute the gradients for parameters.
          loss.backward()

          # Clip the gradient norms for stable training.
          grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

          # Update the parameters with computed gradients.
          optimizer.step()

          # Compute the accuracy for current batch.
          acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

          # Record the loss and accuracy.
          train_loss.append(loss.item())
          train_accs.append(acc)

      train_loss = sum(train_loss) / len(train_loss)
      train_acc = sum(train_accs) / len(train_accs)

      # Print the information.
      print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

      # ---------- Validation ----------
      # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
      model.eval()

      # These are used to record information in validation.
      valid_loss = []
      valid_accs = []

      # Iterate the validation set by batches.
      for batch in tqdm(valid_loader):

          # A batch consists of image data and corresponding labels.
          imgs, labels = batch
          #imgs = imgs.half()

          # We don't need gradient in validation.
          # Using torch.no_grad() accelerates the forward process.
          with torch.no_grad():
              logits = model(imgs.to(device))

          # We can still compute the loss (but not the gradient).
          loss = criterion(logits, labels.to(device))

          # Compute the accuracy for current batch.
          acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

          # Record the loss and accuracy.
          valid_loss.append(loss.item())
          valid_accs.append(acc)
          #break

      # The average loss and accuracy for entire validation set is the average of the recorded values.
      valid_loss = sum(valid_loss) / len(valid_loss)
      valid_acc = sum(valid_accs) / len(valid_accs)

      # Print the information.
      print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

      scheduler.step(valid_acc)
      # update logs
      if valid_acc > best_acc:
          with open(f"./{_exp_name}_log.txt","a"):
              print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
      else:
          with open(f"./{_exp_name}_log.txt","a"):
              print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


      # # save models
      # if valid_acc > best_acc:
      #     print(f"Best model found at epoch {epoch}, saving model")
      #     torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
      #     best_acc = valid_acc
      #     stale = 0
      # else:
      #     stale += 1
      #     if stale > patience:
      #         print(f"No improvment {patience} consecutive epochs, early stopping")
      #         break
    # Save the best model for this fold
      if valid_acc > best_acc:
          print(f"Best model found at epoch {epoch}, saving model for fold {fold + 1}")
          torch.save(model.state_dict(), f"fold_{fold + 1}_best.ckpt")
          best_acc = valid_acc
          stale = 0
          best_model_state_dict = model.state_dict()
      else:
          stale += 1
          if stale > patience:
              print(f"No improvement {patience} consecutive epochs, early stopping for fold {fold + 1}")
              break

  if best_acc > best_overall_acc:
    print(f"New best model found for all folds, saving model")
    torch.save(model.state_dict(), f"best_overall.ckpt")
    best_overall_acc = best_acc




Fold 1/3


  0%|          | 0/103 [00:00<?, ?it/s]

[ Train | 001/030 ] loss = 2.22922, acc = 0.20297


  0%|          | 0/52 [00:00<?, ?it/s]

[ Valid | 001/030 ] loss = 2.16260, acc = 0.22877
[ Valid | 001/030 ] loss = 2.16260, acc = 0.22877 -> best
Best model found at epoch 0, saving model for fold 1


...

[ Valid | 029/030 ] loss = 1.88935, acc = 0.36418
[ Valid | 029/030 ] loss = 1.88935, acc = 0.36418


  0%|          | 0/103 [00:00<?, ?it/s]

[ Train | 030/030 ] loss = 1.45365, acc = 0.50684


  0%|          | 0/52 [00:00<?, ?it/s]

[ Valid | 030/030 ] loss = 1.79274, acc = 0.41947
[ Valid | 030/030 ] loss = 1.79274, acc = 0.41947


In [ ]:
# After all folds, select the best model and train it on the entire dataset
if best_model_state_dict is not None:
    print("Training the best model on the entire dataset...")
    model.load_state_dict(best_model_state_dict)
    full_train_loader = DataLoader(train_set,batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

    # Reset optimizer and scheduler (if needed)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', patience=patience, factor=0.5, verbose=True)

    for epoch in range(n_epochs):
        # Training on the entire dataset
        model.train()
        train_loss = []
        train_accs = []

        for batch in tqdm(full_train_loader):
            imgs, labels = batch
            optimizer.zero_grad()
            logits = model(imgs.to(device))
            loss = criterion(logits, labels.to(device))
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
            train_accs.append(acc)

        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)
        print(f"[Train | Epoch {epoch + 1}/{n_epochs}] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

        scheduler.step(train_acc)

    torch.save(model.state_dict(), "best_overall.ckpt")
    print("Training on the entire dataset completed. Best overall model saved.")

Training the best model on the entire dataset...


  0%|          | 0/155 [00:00<?, ?it/s]

[Train | Epoch 1/30] loss = 1.55666, acc = 0.47147


  0%|          | 0/155 [00:00<?, ?it/s]

[Train | Epoch 2/30] loss = 1.44175, acc = 0.51679


...

[Train | Epoch 28/30] loss = 0.13358, acc = 0.95877


  0%|          | 0/155 [00:00<?, ?it/s]

[Train | Epoch 29/30] loss = 0.09754, acc = 0.97002


  0%|          | 0/155 [00:00<?, ?it/s]

[Train | Epoch 30/30] loss = 0.10691, acc = 0.96746
Training on the entire dataset completed. Best overall model saved.


# Dataloader for test

In [ ]:
# Construct test datasets.
# The argument "loader" tells how torchvision reads the data.
test_set = FoodDataset("./data/test", tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# Testing and generate prediction CSV

In [ ]:
model_best = Classifier().to(device)
#model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
model_best.load_state_dict(torch.load("best_overall.ckpt"))
model_best.eval()
prediction = []
with torch.no_grad():
    for data,_ in tqdm(test_loader):
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()

  0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
# create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(len(test_set))]
df["Label"] = prediction
df.to_csv("submission_Final_.csv",index = False)